In [37]:
from scipy.signal import lfilter, lfilter_zi
import numpy as np
import pyaudio as pa
from scipy.io import wavfile
from time import time
from IPython.display import Audio

In [99]:
def tofloat32(x): return (x / np.abs(x).max()).astype(np.float32)

# No olvidarse de correr la última línea si prendiste el PyAudio

In [ ]:
p = pa.PyAudio()

In [68]:
fs, data = wavfile.read('speech.wav')
data = tofloat32(data)
print(len(data))

62430


In [4]:
def comb_filter(fs : int, data, tau : float, T60 : float, zi = None) -> tuple[np.array, np.array]:
    N = int(np.round(tau * fs, 0))
    a = np.zeros(N + 1)
    a[[0, N]] = [1, 10 ** (-3 * tau / T60)]
    b = [1.0]
    if zi is None: zi = np.zeros(lfilter_zi(b, a).size)
        
    return lfilter(b, a, data, zi = zi)

In [5]:
size = data.size
y = data
chunksize, tau, T60 = 1024, 50e-3, 2

stream = p.open(channels = 1, format = pa.paFloat32, rate = fs, output = True)
stream.start_stream()

y_chunk = y[ : chunksize]
y_filtered, zi = comb_filter(fs, y_chunk, tau, T60)
y_filtered = tofloat32(y_filtered).tobytes()

i = 1
while len(y_filtered) > 0:
    stream.write(y_filtered)
    y_chunk = y[i * chunksize : (i + 1) * chunksize]
    y_filtered, zi = comb_filter(fs, y_chunk, tau, T60, zi)
    if y_filtered.size and np.count_nonzero(y_filtered): y_filtered = tofloat32(y_filtered).tobytes()
    i += 1
    
stream.stop_stream()
stream.close()
p.terminate()

Lo de arriba ya está bien

In [24]:
def allPass_filter(fs, data, tau, T60, zi = None):
    N = int(np.round(tau*fs, 0))
    g = 10**(-3*tau/T60)
    b = np.zeros(N+1)
    a = np.zeros(N+1)
    
    b[0] = -g
    b[N] = 1
    a[0] = 1
    a[N] = -g
    
    if zi is None: ni = np.zeros(lfilter_zi(b, a).size)
    
    return signal.lfilter(b, a, data, zi = zi)

In [36]:
def shunt_comb_filter(fs, data, tau, T60):
    size = len(tau)

    y, zi = [], []
    y_aux, zi_aux = [], []
    for i in range(size):
        y_aux, zi_aux = comb_filter(fs, data, tau[i], T60[i])
        y.append(y_aux)
        zi.append(zi_aux)
    return y, zi
    

In [37]:
p = pa.PyAudio()
stream = p.open(channels = 1, format = pa.paFloat32, rate = fs, output = True)
stream.start_stream()

chunksize = 1024
tau1, tau2, tau3, tau4 = 91e-3, 77e-3, 101e-3, 112e-3#901e-3, 778e-3, 1011e-3, 1123e-3
g_1, g_2, g_3, g_4 = 0.805, 0.827, 0.783, 0.764
T60_1, T60_2, T60_3, T60_4 = (-3*tau1/np.log10(g_1)), (-3*tau2/np.log10(g_2)), (-3*tau3/np.log10(g_3)), (-3*tau4/np.log10(g_4))
tau1_ap, tau2_ap = 100e-3, 100e-3/3
g_ap = 0.7
T60_ap1, T60_ap2 = (-3*tau1_ap/np.log10(0.7)), (-3*tau2_ap/np.log10(0.7))
y = data

y_chunk = y[ : chunksize]

y_comb , zi = shunt_comb_filter(fs, y_chunk, [tau1, tau2, tau3, tau4], [T60_1, T60_2, T60_3, T60_4])

y_allPass1, zi_alPass1 = allPass_filter(fs, np.sum(y_comb, axis=0))

for i in range(len(y_comb)):
    y_comb[i] = tofloat32(y_comb[i]).tobytes()

    
    

#i = 1
#while len(y_comb[1]) > 0:
#    #stream.write(y_filtered1)
#    y_chunk = y[i * chunksize : (i + 1) * chunksize]
#    y_filtered1, zi1 = comb_filter(fs, y_chunk, tau, T60, zi)
#    if y_filtered1.size and np.count_nonzero(y_filtered1): y_filtered1 = tofloat32(y_filtered1).tobytes()
#    i += 1



stream.stop_stream()
stream.close()
p.terminate()

N:  5964
N:  5046
N:  6619
N:  7340
N:  3277


ValueError: setting an array element with a sequence.

Lo de abajo estoy ahora

In [21]:
def transfer_function(fs, tau, T60, filter_type):
    N = int(np.round(tau*fs, 0))
    g = 10**(-3*tau/T60)
    b = np.zeros(N + 1)
    a = np.zeros(N+1)
    
    if filter_type == 'comb_filter':
        b[0] = 1.0
    else:
        b[0] = -g
        b[N] = 1.0
    a[0] = 1.0
    a[N] = -g
    return np.poly1d(b), np.poly1d(a)


def comb_filter(fs : int, data, tau : float, T60 : float, zi = None) -> tuple[np.array, np.array]:
    N = int(np.round(tau * fs, 0))
    a = np.zeros(N + 1)
    a[[0, N]] = [1, 10 ** (-3 * tau / T60)]
    b = [1.0]
    if zi is None: zi = np.zeros(max(len(b), a.size) - 1)
    print(len(b), a.size)    
    return lfilter(b, a, data, zi = zi)

In [78]:
def create_tf(fs, tau, T60, filter_type):
    
    h = []
    for i, t in enumerate(tau):
        num, den = transfer_function(fs, t, T60[i], filter_type)
        h.append([num, den])
    
    if filter_type == 'comb_filter':
        
        num, denom = 0, 1
        for i, transfer in enumerate(h):
            denom *= transfer[1] 
            temp_num = transfer[0]
            
            for j, other_div in enumerate(h):
                if j == i: continue
                temp_num *= other_div[1]
                
            num += temp_num
        
    elif filter_type == 'ap_filter':
        num, denom = h[0][0] * h[1][0], h[0][1] * h[1][1]
        
    return num.coef, denom.coef


In [93]:
chunksize = 1024
tau_comb = np.array([50e-3, 40e-3, 60e-3])#[9.1e-3, 7.7e-3, 10.1e-3, 11.2e-3]
g_comb = np.array([0.5])
T60_comb = [2, 2.5, 1.5]#-3*tau_comb / np.log10(g_comb) #[(-3*tau_comb[0]/np.log10(g_1)), (-3*tau_comb[1]/np.log10(g_2)), (-3*tau_comb[2]/np.log10(g_3)), (-3*tau_comb[3]/np.log10(g_4))]
tau_ap = np.array([100e-3, 100e-3/3])
g_ap = 0.7
T60_ap = -3*tau_ap/np.log10(g_ap)
y = data

In [96]:
#stream = p.open(channels = 1, format = pa.paFloat32, rate = fs, output = True)
#stream.start_stream()


y_chunk = y[ : chunksize]

h_comb = create_tf(fs, tau_comb, T60_comb, 'comb_filter')
h_ap = create_tf(fs, tau_ap, T60_ap, 'ap_filter')
#h_ap = [1, 1]

h_tot = [np.polymul(h_comb[0], h_ap[0]), np.polymul(h_comb[1], h_ap[1])]
print(h_tot)

y_filt, zi = lfilter(h_tot[0], h_tot[1], y_chunk, zi = np.zeros(-1 + max(len(h_tot[0]), len(h_tot[1]))))

res = np.zeros(y.size)

res[ : chunksize] = y_filt

y_filt = tofloat32(y_filt).tobytes()
i = 1

while len(y_filt) > 0 and i < 400:
    #stream.write(y_filt)
    y_chunk = y[i * chunksize : (i + 1) * chunksize]
    y_filt, zi = lfilter(h_tot[0], h_tot[1], y_chunk, zi = zi) 
    res[i * chunksize : (i + 1) * chunksize] = y_filt
    if not i%1000: print(i)
    #if y_filt.size and np.count_nonzero(y_filt): y_filt = tofloat32(y_filt).tobytes()
    i += 1
    
#stream.stop_stream()
#stream.close()

[array([1.47, 0.  , 0.  , ..., 0.  , 0.  , 0.  ]), array([ 1.        ,  0.        ,  0.        , ...,  0.        ,
        0.        , -0.28002453])]


In [97]:
Audio(res, rate = fs)

# No olvidarse de correr esto si prendiste el PyAudio

In [77]:
p.terminate()